## Miniproject 3 - Decision Trees, by Shawn Nassabi (san7522)

In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt


In [4]:
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
iris_df= iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2})


In [5]:
print(iris_df.shape) # 150, 5, 5th is the target
iris_df.head
#

(150, 5)


<bound method NDFrame.head of      sepal_length  sepal_width  petal_length  petal_width  class
0             5.1          3.5           1.4          0.2      0
1             4.9          3.0           1.4          0.2      0
2             4.7          3.2           1.3          0.2      0
3             4.6          3.1           1.5          0.2      0
4             5.0          3.6           1.4          0.2      0
..            ...          ...           ...          ...    ...
145           6.7          3.0           5.2          2.3      2
146           6.3          2.5           5.0          1.9      2
147           6.5          3.0           5.2          2.0      2
148           6.2          3.4           5.4          2.3      2
149           5.9          3.0           5.1          1.8      2

[150 rows x 5 columns]>

## Decision tree and its methods:

In [6]:
class node:
    def __init__(self, col=None, crit=None):
        self.col = col
        self.crit = crit
        self.left = None
        self.right = None
        self.isLeaf = False
        self.res = None



class decision_tree:
    def __init__(self, minLimit):
        self.root = node()
        self.minLimit = minLimit
        self.minAmount = None

    def get_entropy(self, targetCol):
        entropy = 0
        total_length = len(targetCol)
        target_counts = targetCol.value_counts()
        probabilities = target_counts/total_length
        for p in probabilities:
            if p != 0:
                entropy += p * np.log2(p)
    
        return -entropy
        
    def get_info_gain(self, colName, crit, parent, target):
        leftSplit = parent[parent[colName] < crit]
        rightSplit = parent[parent[colName] >= crit]
        parent_entropy = self.get_entropy(parent[target])
        left_entropy = self.get_entropy(leftSplit[target])
        right_entropy = self.get_entropy(rightSplit[target])
        length = parent.shape[0]
        IG = parent_entropy - ((leftSplit.shape[0]/length)*left_entropy + (rightSplit.shape[0]/length)*right_entropy)
        return IG

    def get_best_crit(self,sample, colName, target): # Get the best criteria for the given column
        highestIG = 0
        bestCrit = None
        u_crits, u_crits_occ = np.unique(sample[colName], return_counts=True)
        for c in u_crits:
            ig = self.get_info_gain(colName, c, sample, target)
            if ig >= highestIG:
                highestIG = ig
                bestCrit = c
        return bestCrit

    def get_best_col(self, sample, target): # Find the best column and the best criteria for it
        
        highestIG = 0
        bestCol = None
        bestCrit = None
        for col in sample.columns:
            if col == target:
                continue
            crit = self.get_best_crit(sample, col, target)
            ig = self.get_info_gain(col, crit, sample, target)
            if ig >= highestIG:
                highestIG = ig
                bestCol = col
                bestCrit = crit
        return bestCol, bestCrit

    def train(self, curNode, sample, target):
        #print(sample.shape[0])
        if sample.shape[0] <= self.minAmount: # Min number of rows reached, so it is a leaf
            curNode.isLeaf = True
            unique_classes, unique_class_occurrences = np.unique(sample[target], return_counts=True)
            bestClass = None
            highestOcc = 0
            for class_value, occurrence in zip(unique_classes, unique_class_occurrences): # Assign most occuring class as the bestClass
                if occurrence > highestOcc:
                    highestOcc = occurrence
                    bestClass = class_value
            curNode.res = bestClass
            return
            
            
        if self.get_entropy(sample[target]) == 0: # Entropy is 0 so it is a leaf
            curNode.isLeaf = True
            curNode.res = sample[target].mode().iloc[0] # assign the res as the only target value that is left
            return

        if not curNode.isLeaf:
            col, crit = self.get_best_col(sample, target)
            curNode.col = col
            curNode.crit = crit
            leftSplit = sample[sample[col] < crit]
            rightSplit = sample[sample[col] >= crit]
            curNode.left = node()
            curNode.right = node()
            self.train(curNode.left, leftSplit, target)
            self.train(curNode.right, rightSplit, target)

        
        
    def fit(self, data, target):
        length = data.shape[0]
        self.minAmount = length * (self.minLimit/100) # Calculate minimum amount threshold for leaf
        self.train(self.root, data, target)

    def predict(self, inputRow):
        found = False
        ptr = self.root
        while not found:
            if ptr.isLeaf: # If current ptr is a leaf then you have arrived at a prediction
                found = True
                break
            if inputRow[ptr.col] < ptr.crit: # If less than current ptr's criteria then go left
                ptr = ptr.left
            else:
                ptr = ptr.right 
        return ptr.res

    def score(self, inputs, target):
        length = inputs.shape[0]
        correctCount = 0
        for i in range(length):
            prediction = self.predict(inputs.iloc[i])
            if prediction == inputs[target][i]:
                correctCount += 1
        return correctCount / length



        
# irisTree = decision_tree(5)
# irisTree.fit(iris_df, "class")
# prediction = irisTree.predict(iris_df.iloc[149])
# print(prediction)

## Training on iris dataset, using KFold:

In [7]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

IRIS_LIMITS = [5, 10, 15, 20] # Threshold limits to test
iris_limit_means = []
iris_limit_std = []

for limit in IRIS_LIMITS:
    accuracy_scores = []
    for train_index, test_index in kf.split(iris_df): # Split iris dataset for kfold

        iris_train, iris_test = iris_df.iloc[train_index], iris_df.iloc[test_index]

        tree = decision_tree(limit) # Create decision tree with appropriate limit
        target = iris_train.columns[-1]
        tree.fit(iris_train, target) # fit the tree using the training data and last column as target

        iris_pred = [tree.predict(row) for _, row in iris_test.iterrows()] # get predictions array

        accuracy = accuracy_score(iris_test[target], iris_pred) # calculate accuracy scores
        accuracy_scores.append(accuracy)

    accuracy_mean = np.mean(accuracy_scores)
    accuracy_std = np.std(accuracy_scores)
    iris_limit_means.append(accuracy_mean)
    iris_limit_std.append(accuracy_std)

# print(IRIS_LIMITS)
# print(limit_means)
# print(limit_std)

iris_table = pd.DataFrame({
    'Min Thresholds': IRIS_LIMITS,
    'Accuracy Means': iris_limit_means,
    'Std Deviations': iris_limit_std
})
print(iris_table)
    

   Min Thresholds  Accuracy Means  Std Deviations
0               5        0.953333        0.052068
1              10        0.953333        0.052068
2              15        0.953333        0.052068
3              20        0.953333        0.052068


## Spambase Dataset:

In [8]:
spambase_df = pd.read_csv("spambase.csv")

unique_column_names = ["col_" + str(i) for i in range(len(spambase_df.columns) - 1)] + ["class"]


# Assign the unique names to the columns because my tree implementation uses column names
spambase_df.columns = unique_column_names
spambase_df.head

<bound method NDFrame.head of       col_0  col_1  col_2  col_3  col_4  col_5  col_6  col_7  col_8  col_9  \
0      0.21   0.28   0.50    0.0   0.14   0.28   0.21   0.07   0.00   0.94   
1      0.06   0.00   0.71    0.0   1.23   0.19   0.19   0.12   0.64   0.25   
2      0.00   0.00   0.00    0.0   0.63   0.00   0.31   0.63   0.31   0.63   
3      0.00   0.00   0.00    0.0   0.63   0.00   0.31   0.63   0.31   0.63   
4      0.00   0.00   0.00    0.0   1.85   0.00   0.00   1.85   0.00   0.00   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
4595   0.31   0.00   0.62    0.0   0.00   0.31   0.00   0.00   0.00   0.00   
4596   0.00   0.00   0.00    0.0   0.00   0.00   0.00   0.00   0.00   0.00   
4597   0.30   0.00   0.30    0.0   0.00   0.00   0.00   0.00   0.00   0.00   
4598   0.96   0.00   0.00    0.0   0.32   0.00   0.00   0.00   0.00   0.00   
4599   0.00   0.00   0.65    0.0   0.00   0.00   0.00   0.00   0.00   0.00   

      ...  col_48  col_49  col_50

## Training tree on Spambase and using KFold:

In [9]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

SB_LIMITS = [5, 10, 15, 20, 25] # Threshold limits to test
SB_limit_means = []
SB_limit_std = []

for limit in SB_LIMITS:
    accuracy_scores = []
    for train_index, test_index in kf.split(spambase_df): # Split spambase dataframe for kfold

        spambase_train, spambase_test = spambase_df.iloc[train_index], spambase_df.iloc[test_index]

        tree = decision_tree(limit) # Create decision tree with appropriate limit
        target = spambase_train.columns[-1]
        tree.fit(spambase_train, target) # fit the tree using the training data and last column as target

        spambase_pred = [tree.predict(row) for _, row in spambase_test.iterrows()] # get predictions array

        accuracy = accuracy_score(spambase_test[target], spambase_pred) # calculate accuracy scores
        accuracy_scores.append(accuracy)

    accuracy_mean = np.mean(accuracy_scores)
    accuracy_std = np.std(accuracy_scores)
    SB_limit_means.append(accuracy_mean)
    SB_limit_std.append(accuracy_std)

print("DONE")

DONE


In [12]:
SB_table = pd.DataFrame({
    'Min Thresholds': SB_LIMITS,
    'Accuracy Means': SB_limit_means,
    'Std Deviations': SB_limit_std
})
print("Spambase results:")
print(SB_table)

Spambase results:
   Min Thresholds  Accuracy Means  Std Deviations
0               5        0.900870        0.017853
1              10        0.888478        0.014323
2              15        0.868261        0.023011
3              20        0.858696        0.020300
4              25        0.827609        0.016529
